In [1]:
import numpy as np
import gzip
import os
import tensorflow as tf
import re

# specify the path to the zipped file.
os.chdir('/mnt/c/Users/amsj1/OneDrive - University of Tennessee/2nd_year/BZAN554_deep_learning/bzan554-group-assignment1')

2021-09-21 15:19:22.671868: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-21 15:19:22.671922: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def parse(path):
    """
    Function to read in the VERY LARGE dataset and yield it as a generator for
    memory efficiency. Takes one argument which is the path to the file being
    read in. This path is set abose using 'os.chdir'.
    """
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)

#######################################################################
# Basically load in some data to make lookup tables for the sequentially
# learning part
#######################################################################
# X_raw = []
lookup_X = []
Y_raw = []
counter = 0
for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'):
    counter += 1
    # X_raw.append(d['title'])
    X_raw = np.array(d['title'])
    Y_raw.append(d['category'])

    X_words = [words for words in str(X_raw).lower().split()]
    lookup_X.extend(X_words)

    if counter == 1000:
        break

# X_words = []
# for titles in X_raw:
#     # split the titles into words and lowercase
#     words = titles.lower().split()
#     for word in words:
#         # extract out words with minimal to no symbols
#         stripped_word = re.sub('[0-9,.,#,\-,&,;,\',(,),/,@,+,!]', '', word)
#         X_words.append(stripped_word)

# X_words = np.unique(np.array(X_words))

# look up for X
# lookup_X = []
# lookup_X.extend([word for word in X_words])

# flatten the Y list of lists
flat_Y = [category for subcat in Y_raw for category in subcat]
# uniquefy it
unique_Y = np.unique(np.array(flat_Y))

# look up for Y
lookup_Y = []
lookup_Y.extend([category for category in unique_Y])

In [3]:
#######################################################################
# Set up the model architecture
#######################################################################
inputs = tf.keras.layers.Input(shape = (len(lookup_X),))
hidden1 = tf.keras.layers.Dense(
    units = len(lookup_X),
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = len(lookup_Y),
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
output = tf.keras.layers.Dense(
    units = len(unique_Y),
    activation = 'sigmoid',
    name = 'output'
)(hidden2)

#######################################################################
# create & compile the model
#######################################################################
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.001)
)

2021-09-21 15:19:26.417928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-21 15:19:26.418033: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-21 15:19:26.418198: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-A5OMSR4): /proc/driver/nvidia/version does not exist
2021-09-21 15:19:26.418492: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#######################################################################
# Big boy model learning time
#######################################################################
for i in range(10): # number of epochs
    counter = 0
    for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'): # instances
        counter += 1
        X_raw = np.array(d['title'])
        Y_raw = np.array(d['category'])

        ###############################################
        # logic for the X hot-one-encoded stuff
        ###############################################
        # split the titles into words and lowercase
        # words = word.lower().split()
        X_words = [words for words in str(X_raw).lower().split()]

        # Get final binary stuff for X
        X_indices = np.where(np.isin(lookup_X, X_words))
        X_final = np.zeros(len(lookup_X))
        X_final[X_indices] = 1
        X_final = X_final.reshape(1, len(X_final))

        ###############################################
        # logic for the Y hot-one-encoded stuff
        ###############################################
        Y_indices = np.where(np.isin(lookup_Y, unique_Y))
        Y_final = np.zeros(len(lookup_Y))
        Y_final[Y_indices] = 1
        Y_final = Y_final.reshape(1, len(Y_final))

        ###############################################
        # Finallllyyyyy fit the freakin model
        ###############################################
        model.fit(x=X_final,y=Y_final,epochs=1,batch_size=1)

        if counter == 1000:
            break

2021-09-21 15:19:27.417706: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 [==============================] - 0s 313ms/step - loss: 0.4498


KeyboardInterrupt: 

In [8]:
yhat = model.predict(x=X_final)
yhat.min()

0.31643456